In [ ]:
%matplotlib inline

from __future__ import absolute_import #style_guide
from __future__ import division        #style_guide
from __future__ import print_function  #style_guide

import os  #file access
import csv #output results
import time  #timing epochs
from datetime import timedelta #timing epochs

import math #might not need
import numpy as np #math/arrays
import tensorflow as tf #NN
import tflearn #might not need - NN if I do
import matplotlib.pyplot as plt #for plotting data

import PIL
from PIL import Image #image processing
from zipfile import ZipFile #File processing
from tqdm import tqdm_notebook as tqdm #Progress Bar

from sklearn.utils import shuffle  #Shuffling training data
from sklearn.preprocessing import LabelBinarizer  #creating label set

# Constants
## image/general

In [ ]:
img_width = 32
img_height = 18
img_size_flat = img_height * img_width

img_shape = (img_width, img_height)
arr_shape = (img_height, img_width)

#RGB
num_channels = 3

#Number of classes. In order(minus image):
csv_header = ['image', 'ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
num_classes = 8
learning_rate = 1e-4

# General Helper Functions

In [ ]:
def normalize_pix(pixels):
    norm = np.array(pixels.shape)
    a=.1
    b=.9
    x_max = 255
    x_min = 0
    
    norm = a + ((pixels - x_min)*(b-a)/(x_max-x_min))
    return norm

In [ ]:
def uncompress_features_labels(file):
    global img_size
    features = []
    labels   = []
    current_label = ""
    
    with ZipFile(file) as zf:
        #Progress Bar
        filenames_pbar = tqdm(zf.namelist(), unit='files')
        #Features and labels
        for i, filename in enumerate(filenames_pbar):
            if not filename.endswith('/') and filename.endswith('.jpg'):
                with zf.open(filename) as image_file:
                    image = Image.open(image_file).resize(img_shape,resample = PIL.Image.LANCZOS)
                    image_arr = normalize_pix(np.array(image, dtype=np.float32))
                    if filename[6:9] != current_label:
                        current_label = filename[6:9] if "/" in filename[6:10] else filename[6:11]

                    features.append(image_arr)
                    labels.append(current_label)
                       
    return np.array(features),np.array(labels)

In [ ]:
def data_prep(file):
    features, labels = uncompress_features_labels(file)
    
    #binarize for use with class_vector
    labels = binarize_labels(labels)
    
    #shuffle data
    features,labels = shuffle(features, labels)

    #break into train and validation data
    #3527 training_samples
    train_features  = features[:3527]
    train_labels    = labels[:3527]

    #250 val_samples
    valid_features  = features[3527:]
    valid_labels    = labels[3527:]
    
    return (train_features,train_labels),(valid_features,valid_labels)

# Neural Network Helper Functions

In [ ]:
# create matrix of weights for given shape from Gaussian dist. with stddev of .05,
# truncated so anything over st. dev. 2 is re-sampled
def new_weights(shape,name='W'):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name=name)

In [ ]:
# Creates new biases for a NN layer
def new_biases(length,name='B'):
    return tf.Variable(tf.zeros(shape=[length]), name=name)

In [ ]:
# takes input, weight matrix, and name, and returns a 2d convolution tensor
# with stride 1 and padding SAME, used in new_inception_layer function
def conv2d_s1(x,W,name='conv_s1'):
    return tf.nn.conv2d(input=x,
                        filter=W,
                        strides=[1,1,1,1],
                        padding='SAME', name=name)

In [ ]:
# returns a max_pooled version of input x with size 3x3 and stride 1
def max_pool_3x3_s1(x):
    return tf.nn.max_pool(x,ksize=[1,3,3,1], strides=[1,1,1,1], padding='SAME')

In [ ]:
# Creates an inception module, with 1x1 convolution that feeds seperately into the output, a 3x3 convolution filter,
# and a 5x5 convolution filter. In addition, a 3x3 max_pool is of the input is fed into another 1x1 filter, and all
# 4 are concatenated, to create the final output

def new_inception_layer(x, depth1x1, num_filters,
                        num_channels,name='Incep'):
    with tf.name_scope(name):
        # connected to input
        # direct 1x1 convolution
        W_conv_1x1_1_name = 'W_1x1_1_{}x{}x{}x{}'.format(1,1,num_channels,num_filters)
        W_conv_1x1_1  = new_weights([1,1,num_channels,num_filters],name=W_conv_1x1_1_name) #1x1x3x32
        b_conv_1x1_1 = new_biases(num_filters,name='B_{}'.format(num_filters))
        tf.summary.histogram(W_conv_1x1_1_name,W_conv_1x1_1)
        tf.summary.histogram('B_{}'.format(num_filters),b_conv_1x1_1)

        # connected to input
        # dim_reduc before 3x3 convolution
        W_conv_1x1_2_name = 'W_1x1_2_{}x{}x{}x{}'.format(1,1,num_channels,depth1x1)
        W_conv_1x1_2 = new_weights([1,1,num_channels,depth1x1],name=W_conv_1x1_2_name)
        b_conv_1x1_2 = new_biases(depth1x1,name='B_{}'.format(depth1x1))
        tf.summary.histogram(W_conv_1x1_2_name,W_conv_1x1_2)
        tf.summary.histogram('B_{}'.format(depth1x1),b_conv_1x1_2)
        
        # connected to inputs
        # dim reduc before5x5 convolution
        W_conv_1x1_3_name = 'W_1x1_3_{}x{}x{}x{}'.format(1,1,num_channels,depth1x1)
        W_conv_1x1_3 = new_weights([1,1,num_channels,depth1x1],name=W_conv_1x1_3_name)
        b_conv_1x1_3 = new_biases(depth1x1,name='B_{}'.format(depth1x1))
        tf.summary.histogram(W_conv_1x1_3_name,W_conv_1x1_3)
        tf.summary.histogram('B_{}'.format(depth1x1),b_conv_1x1_3)

        # connected to 1x1_2
        W_conv_3x3_name = 'W_3x3_{}x{}x{}x{}'.format(3,3,depth1x1,num_filters)
        W_conv_3x3 = new_weights([3,3,depth1x1,num_filters],name=W_conv_3x3_name)
        b_conv_3x3  = new_biases(num_filters,name='B_{}'.format(num_filters))
        tf.summary.histogram(W_conv_3x3_name,W_conv_3x3)
        tf.summary.histogram('B_{}'.format(num_filters),b_conv_3x3)
            
        # connected to 1x1_3
        W_conv_5x5_name = 'W_5x5_{}x{}x{}x{}'.format(5,5,depth1x1,num_filters)
        W_conv_5x5 = new_weights([5,5,depth1x1,num_filters],name=W_conv_5x5_name)
        b_conv_5x5  = new_biases(num_filters,name='B_{}'.format(num_filters))
        tf.summary.histogram(W_conv_5x5_name,W_conv_5x5)
        tf.summary.histogram('B_{}'.format(num_filters),b_conv_5x5)

        # connected to max_pool
        W_conv_1x1_4_name = 'W_1x1_4_{}x{}x{}x{}'.format(1,1,num_channels,depth1x1)
        W_conv_1x1_4 = new_weights([1,1,num_channels,depth1x1],name=W_conv_1x1_4_name)
        b_conv_1x1_4 = new_biases(depth1x1,name='B_{}'.format(depth1x1))
        tf.summary.histogram(W_conv_1x1_4_name,W_conv_1x1_4)
        tf.summary.histogram('B_{}'.format(depth1x1),b_conv_1x1_4)

        conv_1x1_1 = conv2d_s1(x,W_conv_1x1_1)+b_conv_1x1_1
        conv_1x1_2 = tf.nn.relu(conv2d_s1(x,W_conv_1x1_2)+b_conv_1x1_2)
        conv_1x1_3 = tf.nn.relu(conv2d_s1(x,W_conv_1x1_3)+b_conv_1x1_3)
        conv_3x3 = conv2d_s1(conv_1x1_2,W_conv_3x3) + b_conv_3x3
        conv_5x5 = conv2d_s1(conv_1x1_3,W_conv_5x5) + b_conv_5x5
        max_pool = max_pool_3x3_s1(x)
        conv_1x1_4 = conv2d_s1(max_pool,W_conv_1x1_4) + b_conv_1x1_4

        inception = tf.nn.relu(tf.concat(values=[conv_1x1_1,conv_3x3,conv_5x5,conv_1x1_4],axis=3))
    return inception

In [ ]:
#Defines a 2d convolution tensor layer
#Has ReLU and pooling built in
def new_conv_layer(input,
                   num_input_channels,
                   filter_size,
                   num_filters,
                   filter_stride=1,
                   use_pooling=2,
                   name='Conv'):
    
    with tf.name_scope(name):
        #define the shape for weight shape
        shape = [filter_size, filter_size, num_input_channels, num_filters]

        #define matrix of weights
        weights_name = 'W_{}x{}x{}x{}'.format(filter_size, filter_size, num_input_channels, num_filters)
        weights = new_weights(shape=shape,name=weights_name)
        tf.summary.histogram(weights_name,weights)

        #define vector of biases
        biases  = new_biases(length=num_filters,name='B_{}'.format(num_filters))
        tf.summary.histogram('B_{}'.format(num_filters),biases)
        #define actual tensorflow cn layers
        layer = tf.nn.conv2d(input=input,
                             filter=weights,
                             strides=[1, filter_stride, filter_stride, 1],
                             padding='SAME')

        #add biases
        layer += biases

        #if use_pooling > 1, does a max pooling with filter size and stride of size use_pooling
        if use_pooling > 1:
            layer = tf.nn.max_pool(value=layer,
                                   ksize=[1, use_pooling, use_pooling, 1],
                                   strides=[1, use_pooling, use_pooling, 1],
                                   padding='SAME')

        #if change to ave_pooling, switch order of ReLU and pooling!
        layer = tf.nn.relu(layer)
    
    return layer, weights

In [ ]:
#Flattens the input layer for use with FC NN layer
def flatten_layer(layer):
    with tf.name_scope('flatten'):
        layer_shape = layer.get_shape()

        #img_height*img_width*num_channels
        num_features = layer_shape[1:4].num_elements()

        #flatten layer to num_images, num_features
        layer_flat = tf.reshape(layer, [-1,num_features])
    
    return layer_flat, num_features

In [ ]:
#Creates a new fully-connected NN layer, used after Convolution layers
def new_fc_layer(input,
                 num_inputs,
                 num_outputs,
                 use_relu=True,
                 name='FC'):
    with tf.name_scope(name):
        #define weights and biases
        weights = new_weights(shape=[num_inputs, num_outputs])
        biases  = new_biases(length=num_outputs)

        #linear combination, tensor object
        layer = tf.matmul(input, weights) + biases

        #if using non-linearity activation
        if use_relu:
            layer = tf.nn.relu(layer)
        
    return layer

In [ ]:
#split sets into batches
def fetch_batches(batch_size,features,labels):
    feature_batches = []
    label_batches   = []
    
    for start_i in range(0,len(features), batch_size):
        end_i = start_i + batch_size
        
        f_batch = features[start_i:end_i]
        l_batch = labels[start_i:end_i]
        
        feature_batches.append(f_batch)
        label_batches.append(l_batch)
        
    return feature_batches, label_batches

In [ ]:
#tag input labels and create a list of length num_classes, with all 0's except the correct class, which is 1
def binarize_labels(label_set):
    encoder = LabelBinarizer()
    encoder.fit(label_set)
    
    label_set = encoder.transform(label_set)
    label_set = label_set.astype(np.float32)
    return label_set

In [ ]:
# For a tensorflow graph
def save_model(file_name):
    saver = tf.train.Saver()
    save_path = saver.save(session, file_name)
    print(save_path)

# Genetic Algorithm Functions

In [ ]:
"""
Builds a chromosome from random
The first gene determines layer types:
    0: Inception module
    1: Convolution layer
    2: Fully Connected(only available for the last layer)
    
    Inception = [layer_type,depth_1x1,filter_depth1,dropout]
        layer_type:   0
        depth_1x1:    2^n, 3 <= n <= 6
        filt_power_1: 2^m, 4 <= m <= 8
        dropout:     True or False
        
    Convolution = [layer_type,filter_size,num_filters,filter_stride,pool_size,dropout]
        layer_type:   1
        filter_size:  2*n+1, 2 <= n <= 4
        num_filters:  2^m,   3 <= m <= 8
        filter_stride:j,     1 <= j <= n
        pool_size:    k,     0 <= k <= 3, does not use pooling if k < 2
        dropout:      True or False
        
    Fully Connected = [layer_type,layer_size,dropout]
        layer_type: 2
        layer_size: 6 <= n <= 9, 2^n
        dropout:    True or False
"""
import random
random.seed()
# creates a chromosome with chr_size layers,
def create_new_chromosome(chr_size=5):
    # return list of genes
    chromosome = []
    
    for i in range(chr_size):
        # first 4 layers are only inception or conv
        types = 1
        
        # last layer could be fully connected
        if i == chr_size-1:
            types = 2
            
        layer_type = random.randint(0,types)
        
        # inception layer
        if layer_type == 0:
            depth_1x1 = 2 ** (random.randint(3,6))
            filter_depth1 = 2 ** random.randint(4,8)
            dropout = True if random.randint(0,1) == 1 else False
            
            gene = [layer_type,depth_1x1,filter_depth1,dropout]
            
        # convolution layer
        elif layer_type == 1:
            filter_size_n = random.randint(2,4)
            filter_size   = 2 * filter_size_n + 1
            num_filters   = 2 ** random.randint(3,8)
            filter_stride = random.randint(1,filter_size_n-1)
            
            pool_size = random.randint(0,3)
            
            dropout = True if random.randint(0,1) == 1 else False
            
            gene = [layer_type,filter_size,num_filters,filter_stride,pool_size,dropout]
            
        # fully connected layer
        else:
            layer_size = 2 ** random.randint(6,9)
            dropout = True if random.randint(0,1) == 1 else False
            gene = [layer_type,layer_size,dropout]
            
        chromosome.append(gene)
        
    return chromosome

In [ ]:
#builds a tensor layer from the provided gene, with the inputs

def build_layer(gene, inputs):
    input_depth = inputs.get_shape().as_list()[3]
    
    # inception module
    if gene[0] == 0:
        depth1x1 = gene[1]
        num_filters = gene[2]
        name = 'incep_d1x1_{}_nf{}'.format(depth1x1,num_filters)
        layer = new_inception_layer(inputs,depth1x1,
                                    num_filters,input_depth,name=name)
    
    # convolution layer
    elif gene[0] == 1:
        filter_size   = gene[1]
        num_filters   = gene[2]
        filter_stride = gene[3]
        pool_size     = gene[4]
        
        name = 'conv_fsize{}_nf{}_fstr{}_pool{}'.format(filter_size,num_filters,
                                                        filter_stride,pool_size)
        layer, _ = new_conv_layer(input=inputs,
                   num_input_channels=input_depth,
                   filter_size=filter_size,
                   num_filters=num_filters,
                   filter_stride=filter_stride,
                   use_pooling=pool_size,
                   name=name)
        
        
    # fully connected layer
    # should ONLY be the last layer
    else:
        num_out = gene[1]
        layer_flat, num_features = flatten_layer(inputs)
        
        name = 'fc_ls{}'.format(num_features)
        layer = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=num_out,
                         use_relu=False,
                         name=name)
        
    return layer

In [ ]:
#mutates a provided gene based on type

def mutate(gene):
    # if its a boolean gene, flip
    if isinstance(gene,bool):
        new = not gene

    # its my pool_size
    elif gene < 4:
        new_pool = random.randint(0,3)
        new = new_pool
    
    # if its an even number(power of 2)
    elif gene%2 == 0:
        # vary power of 2 by one in either direction, or stay the same
        old_power = log(gene)/log(2)
        new_power = random.randint(old_power-1,old_power+1)
        new = 2 ** new_power
        
    # its an odd number
    else:
        # generating odd number via 2n+1, randomly generated n
        # same drift amount as power, could be 1 below or 1 above
        old_n = (gene-1)/2
        new_n = random.randint(old_n-1,old_n+1)
        new = (2 * new_n) + 1
        
    return new

In [ ]:
# takes a list of the 3 top individuals,
# and a mutation_rate that defaults to .5

# calls mutate
def mate(top_dogs,mutation_rate=.5):
    new_gen = []
    # outer loop
    for i in range(len(top_dogs)-1):
        # inner loop, getting so we have every individual mate
        mom = top_dogs[i]
        for j in range(i+1,len(top_dogs)):
            # walking through individuals
            dad = top_dogs[j]
            child = []
            
            # walking through all layers in an individual
            for layer in range(len(mom)):
                new_layer = []
                
                # 50-50 chance we pick mom or dad's layer
                child_layer = mom[layer] if random.random() < .5 else dad[layer]
                
                # not mutating layer type yet
                new_layer.append(child_layer[0])
                
                # for each hyper_parameter, sans the layer_type
                for param in range(1,len(child_layer)):
                    new_param = child_layer[param]
                    
                    # mutate the hyper_param based on mutation rate
                    if random.random() < mutation_rate:
                        new_param = mutate(new_param)
                    new_layer.append(new_param)
                
                # append the newly created layer to the child
                child.append(new_layer)
                
            # we have completed child, now append to list of new generation
            new_gen.append(child)
    
    # 3*(3+1)/2 --> 6
    # elitism keeps the top 2 individuals, concate on new_gen
    # maintains population size of 8
    new_gen.concat(top_dogs[:2])
    return child
                

In [ ]:
def print_pop(population):
    for individual in population:
        print(individual)

In [ ]:
def fitness(pop):
    fit_list = []
    # pop[-1:-2][0] = loss
    # pop[-1:-2][1] = time
    
    # average time usage over the whole population
    ave_time = 0
    for time in pop:
        ave_time += time[-1:-2][1]
    ave_time /= len(pop)
    
    for individual in pop:
        #will punish times over average, and reward times under
        time_cost = -(individual[-1:-2][1]-ave_time)*2
        
        #flip the loss fraction so that smaller losses are good
        loss_fit = 1/individual[-1:-2][0]
        
        #fitness is the addition of these
        #with weight on the loss
        fitness = .7*(loss_fit) + .3*(time_cost)
        
        fit_list.append([fitness,individual])
    
    return sorted(fit_list,key=itemgetter(0),reverse=True)

In [ ]:
# keep top 3 --> mate gives 6
# elitism for top 2 --> gives 8

# Runs genetic algorithm for num_generations of rounds, with specified pop_size

def run_world(train_data,valid_data,pop_size=8,num_generations=10):
    gen_num = 0
    population = []
    best = []

    while gen_num <= num_generations:
        # keep track of generation number for logging
        gen_num += 1
        print("Starting generation {}".format(gen_num))
        
        # generate whole population from random if first generation
        if gen_num == 1:
            print("Generating initial population...")
            for i in range(pop_size):
                population.append(create_new_chromosome(chr_size=5))
            print_pop(population)

        # otherwise, create new population via mating of top 3 individuals
        # assuming pre-sorted based on fitness
        else:
            print("Mating top 3...")
            #randomly generated mutation_rate, from between 0 and .7
            population = mate(population[:3],mutation_rate=random.random(0,.7))
            print_pop(population)

        # print(population)
        # walk through each individual in a population
        for i,individual in enumerate(population):
            layer_count = {0: 0,
                           1: 0,
                           2: 0}
            for gene in individual:
                layer_count[gene[0]] += 1
                
            
            # attempting to implement tensorboard for logging
            log_string = './round_1/gen{}_ind{}_inc{}_conv{}_fc{}'.format(gen_num,i,layer_count[0],
                                                                layer_count[1],layer_count[2])
            
            # takes an individual(NN architecture), sets up a tensor graph according to genes
            # returns the final Validation set-loss, and batch_time for evaluating fitness
            loss,time = evaluate(individual,i,
                                 train_data,valid_data,
                                 log_string)
            print("\tBatch-time: time = {}".format(time))

            # concat loss and time of model on end of individual
            individual += [loss,time]

        # evaluate fitness of population and return sorted, with best at top
        fit_list = fitness(population)

        # copy sorted version of population
        # save the best individual of this generation to its own list for review
        best.append(fit_list[0])
        print("best for gen{} is: ".format(generation))
        print(fit_list[0])
        
        # copy sorted version of population
        # last item element is loss and time, unneeded for new population
        population = [individual[:len(individual)-1] for individual in fit_list]
        
    legend = sorted(fit_list,key=itemgetter(0),reverse=True)
    print("The best of the best is: ")
    print(legend[0])

# Neural Network Build and Run Functions

In [ ]:
# sets up tensor graph and runs for specified num of epochs,

def evaluate(individual,indiv_num,
             train_data,valid_data,
             log_string,epochs=1):
    
    train_features, train_labels = train_data
    valid_features, valid_labels = valid_data
    train_batch_size = 256
    
    tf.reset_default_graph()
    learning_rate=1e-4
    file_writer = tf.summary.FileWriter(log_string)
    
    with tf.name_scope('inputs'):
        x = tf.placeholder(tf.float32, shape=[None, img_height, img_width, num_channels], name='x')
   
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    #create actual tensor graph from the genes in an individual
    model = create_model(individual,x,keep_prob)
    with tf.name_scope('targets'):
        y_true = tf.placeholder(tf.float32, shape=[None,num_classes], name = 'y_true')
        y_true_cls = tf.argmax(y_true, dimension=1)
        
    # apply softmax to turn class prediction into probabilities
    with tf.name_scope('predictions'):
        y_pred = tf.nn.softmax(model)
        y_pred_cls = tf.argmax(y_pred, dimension=1)
        
    with tf.name_scope('loss'):
        mcll_cost = tf.losses.log_loss(y_true,y_pred,epsilon=1e-15)
        tf.summary.scalar('loss',mcll_cost)
        
    with tf.name_scope('train'):
        mcll_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mcll_cost)
    
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(y_pred_cls, y_true_cls)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    #tensors that need to passed to other functions
    test_stuff = (x,y_true,keep_prob,y_pred_cls,mcll_cost)
    
    #merging all of tensorboard elements
    merged = tf.summary.merge_all()
    
    #initialize the batch_time that will be returned
    batch_time = 0
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for e in range(epochs):
            #re-shuffle data on each round
            train_features, train_labels = shuffle(train_features, train_labels)

            #seperate into batches
            batch_data, y_true_batch = fetch_batches(train_batch_size, train_features, train_labels)
            
            #run inside loop over-batches, iterating i inside
            loss = 0
            i=0
            for x_batch, y_true_batch in zip(batch_data, y_true_batch):

                #tensor_feed dictionary for place_holders defined above
                feed_dict = {x: x_batch,
                                   y_true: y_true_batch,
                                   keep_prob: .5}
                
                if e == 0:
                    start_time = time.time()
                summary, batch_loss, _ = sess.run([merged,mcll_cost,
                                                   mcll_optimizer],
                                                   feed_dict=feed_dict)
                if e==0:
                    end_time = time.time()
                    batch_time += end_time - start_time
                
                loss += batch_loss
                file_writer.add_summary(summary)
                if i==0 or i==9:
                    print("num {}: epoch {}: batch{}".format(indiv_num+1,e+1,i+1))
                i+=1
                
            print('Train Loss for Epoch {}: {}'.format(e+1,loss))
            
            #calculate the val_loss and accuracy on validation set after each epoch
            val_loss = print_valid(sess,test_stuff,valid_features, valid_labels,256)
                
        #return val_loss and batch_time for fitness evaluation
        return val_loss, batch_time
    
                                  

In [ ]:
#runs tensor session over validation set, returning class predictions for accuracy, and cost for total loss

def print_valid(sess,test_stuff, features, labels, batch_size):
    #number of test examples
    num_test = len(features)
    x,y_true,keep_prob,y_pred_cls,mcll_cost = test_stuff
    
    cls_pred = np.zeros(shape=num_test, dtype=np.int)
    correct_sum = 0
    i=0
    cost=0
    while i < num_test:
        
        #ending point of batch, either batch size or the end of the test set
        j = min(i+batch_size, num_test)
        
        #batch o' test material
        test_batch  = features[i:j]
        test_lab = labels[i:j]
        
        #feed dat tensor sesh
        feed_dict = {x: test_batch,
                     y_true: test_lab,
                     keep_prob: 1.0}
        
        #the predictions for this batch
        cls_pred[i:j],batch_cost = sess.run([y_pred_cls,mcll_cost], feed_dict=feed_dict)
        
        #move the starting point
        i=j
        cost += batch_cost
    for p,l in zip(cls_pred, labels):
        if l[p] == 1:
            correct_sum += 1
            
    acc = float(correct_sum) / num_test
    
    acc_msg = "\tAccuracy on Valid set: {0:.1%} ({1} / {2})"
    loss_msg = "\tLoss on Valid-Set: {0}"
    print(acc_msg.format(acc, correct_sum, num_test))
    print(loss_msg.format(cost))
    return cost

In [ ]:
# Creates an actual model from the provided chromosome
# calls build_layer

def create_model(chromosome,x,keep_prob):
    # create initial model from input data
    model = build_layer(chromosome[0],x)
    if chromosome[0][-1]:
        model = tf.nn.dropout(model,keep_prob)
    
    # skip the first
    for i in range(1,len(chromosome)):
        gene = chromosome[i]
        model = build_layer(gene,model)
        
        # The last item in each list is bool for whether or not dropout is used
        if gene[-1]:
            model = tf.nn.dropout(model,keep_prob)
            
    # if the last layer was not fully connect,
    # needs to be flattened before sending to fully_connectged output layer
    if gene[0] != 2:
        # if the layer was not flattened, the input for my FC output
        # is num_features returned by my flattening function
        model, num_features = flatten_layer(model)
        
    else:
        # otherwise, it is the size of my previous fully connected layer
        num_features = gene[1]
        
    output_layer = new_fc_layer(input=model,
                         num_inputs=num_features,
                         num_outputs=num_classes,
                         use_relu=False,name='output')
    return output_layer

# Run Stuff

In [ ]:
file = 'train.zip'
train_data,valid_data = data_prep(file)

In [ ]:
#convolution layer, non-matching layers
#inception module, wrong args

run_world(train_data,valid_data)